In [ ]:
import os
import time
import requests
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm

# =========================
# CONFIGURACIÓN INICIAL
# =========================
SPOTIFY_CLIENT_ID = '91c5d14a0f614e6086db1fb0fb658b90'
SPOTIFY_CLIENT_SECRET = '2d33adfece1440f99c2bfd11fc7c102c'
LASTFM_API_KEY = '67d1fc47425dca88529f0ce826fa969f'

# Autenticación con Spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET
))

# Parámetros de búsqueda
genre = "rock"
year_start = 2000
year_end = 2001
query = f"{genre} year:{year_start}-{year_end}"
type_search = "track"
limit = 50
total_tracks = 300  # Puedes poner 300 o más si quieres

# =========================
# OBTENER DATOS DE SPOTIFY
# =========================
data = []

for offset in range(0, total_tracks, limit):
    results = sp.search(q=query, type=type_search, limit=limit, offset=offset)
    tracks = results['tracks']['items']

    for track in tracks:
        data.append({
            "track_name": track['name'],
            "artist_name": track['artists'][0]['name'],
            "album_name": track['album']['name'],
            "release_date": track['album']['release_date'],
            "popularity": track['popularity'],
            "spotify_url": track['external_urls']['spotify']
        })

df = pd.DataFrame(data)

# =========================
# FUNCIÓN PARA LAST.FM
# =========================
def get_lastfm_data(artist, track):
    url = "http://ws.audioscrobbler.com/2.0/"
    params = {
        "method": "track.getInfo",
        "api_key": LASTFM_API_KEY,
        "artist": artist,
        "track": track,
        "format": "json"
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            track_info = data.get('track', {})
            listeners = int(track_info.get('listeners', 0))
            playcount = int(track_info.get('playcount', 0))
            tags = [tag['name'] for tag in track_info.get('toptags', {}).get('tag', [])[:3]]
            return listeners, playcount, tags
        else:
            return None, None, []
    except:
        return None, None, []

# =========================
# AÑADIR DATOS DE LAST.FM
# =========================
tqdm.pandas()
df[['lastfm_listeners', 'lastfm_playcount', 'lastfm_tags']] = df.progress_apply(
    lambda row: pd.Series(get_lastfm_data(row['artist_name'], row['track_name'])),
    axis=1
)

# =========================
# GUARDAR CSV FINAL
# =========================
df.to_csv("spotify_lastfm_tracks.csv", index=False, encoding='utf-8')
print("🎉 Archivo guardado como spotify_lastfm_tracks.csv")


 22%|██▏       | 65/297 [00:16<01:14,  3.13it/s]